# 1. Importing Library

In [1]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import time
import torchvision.models as models
from matplotlib import pyplot as plt
from torchvision import transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# 2. Dataset Loading

### 2.1 Appliying Transformation on Image

In [3]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [4]:
dataset_path = "E:\\3.CBIO\\4.Deep Learning\\1.Car_Damage\\Dataset\\dataset"
dataset = datasets.ImageFolder(root = dataset_path, transform = image_transforms )
len(dataset)

2300

In [5]:
dataset.classes

['F_Breakage', 'F_Crushed', 'F_Normal', 'R_Breakage', 'R_Crushed', 'R_Normal']

In [6]:
num_classes = len(dataset.classes)
num_classes

6

# 3 Train_Test_Split

In [7]:
train_size = int(0.75*len(dataset))
val_size = len(dataset) - train_size

print("train_size ->", train_size, "||", "val_size ->", val_size)

train_size -> 1725 || val_size -> 575


In [8]:
from torch.utils.data import random_split
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle = True)

In [10]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])


In [11]:
images[0].shape

torch.Size([3, 224, 224])

In [12]:
labels[0]

tensor(2)

# 4 Model Building



## Model 1 CNN

In [13]:
class CarClassifierCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), # (16, 224, 224) 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (16, 112, 112),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (32, 56, 56)           
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (64, 28, 28),
            nn.Flatten(),
            nn.Linear(64*28*28, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.network(x)
        return x

In [14]:
images.size(0)

32

In [15]:
len(train_loader.dataset)

1725

In [16]:
def train_model(model, criterion, optimizer, epochs=5):
    start = time.time()
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for batch_num, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            if (batch_num+1) % 10 == 0:
                print(f"Batch: {batch_num+1}, Epoch: {epoch+1}, Loss: {loss.item():0.2f}")
            
            running_loss += loss.item() * images.size(0)
            
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{epochs}], Avg Loss: {epoch_loss:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        all_labels = []
        all_predictions = []
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data,1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                all_labels.extend(labels.cpu().numpy())
                all_predictions.extend(predicted.cpu().numpy())
                
            print(f"*** Validation Accuracy: {100 * correct / total:.2f}% ***")
            
    end = time.time()
    print(f"Execution time: {end - start} seconds")     
    
    return all_labels, all_predictions

In [17]:
# Instantiate the model, loss function, and optimizer
model = CarClassifierCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

all_labels, all_predictions = train_model(model, criterion, optimizer, epochs=5)

Batch: 10, Epoch: 1, Loss: 1.77
Batch: 20, Epoch: 1, Loss: 1.81
Batch: 30, Epoch: 1, Loss: 1.64
Batch: 40, Epoch: 1, Loss: 1.42
Batch: 50, Epoch: 1, Loss: 1.45
Epoch [1/5], Avg Loss: 1.7902
*** Validation Accuracy: 40.17% ***
Batch: 10, Epoch: 2, Loss: 1.41
Batch: 20, Epoch: 2, Loss: 1.43
Batch: 30, Epoch: 2, Loss: 1.36
Batch: 40, Epoch: 2, Loss: 1.17
Batch: 50, Epoch: 2, Loss: 1.25
Epoch [2/5], Avg Loss: 1.2902
*** Validation Accuracy: 48.70% ***
Batch: 10, Epoch: 3, Loss: 1.38
Batch: 20, Epoch: 3, Loss: 1.23
Batch: 30, Epoch: 3, Loss: 1.21
Batch: 40, Epoch: 3, Loss: 0.78
Batch: 50, Epoch: 3, Loss: 1.04
Epoch [3/5], Avg Loss: 1.0863
*** Validation Accuracy: 53.39% ***
Batch: 10, Epoch: 4, Loss: 0.93
Batch: 20, Epoch: 4, Loss: 1.17
Batch: 30, Epoch: 4, Loss: 0.86
Batch: 40, Epoch: 4, Loss: 0.99
Batch: 50, Epoch: 4, Loss: 0.82
Epoch [4/5], Avg Loss: 0.9683
*** Validation Accuracy: 54.61% ***
Batch: 10, Epoch: 5, Loss: 0.79
Batch: 20, Epoch: 5, Loss: 0.77
Batch: 30, Epoch: 5, Loss: 1.10


## Model 2 CNN With Regularazation >> [Dropout, learning Decay, Batch Norm]

In [18]:
class CarClassifierCNNWithRegularization(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1), # (16, 224, 224) 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (16, 112, 112),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (32, 56, 56)           
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0), # (64, 28, 28),
            nn.Flatten(),
            nn.Linear(64*28*28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.network(x)
        return x

In [19]:
model = CarClassifierCNNWithRegularization(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

all_labels, all_predictions = train_model(model, criterion, optimizer,  epochs=10)

Batch: 10, Epoch: 1, Loss: 14.05
Batch: 20, Epoch: 1, Loss: 5.45
Batch: 30, Epoch: 1, Loss: 1.65
Batch: 40, Epoch: 1, Loss: 1.62
Batch: 50, Epoch: 1, Loss: 1.66
Epoch [1/10], Avg Loss: 7.0718
*** Validation Accuracy: 39.65% ***
Batch: 10, Epoch: 2, Loss: 1.45
Batch: 20, Epoch: 2, Loss: 1.65
Batch: 30, Epoch: 2, Loss: 1.38
Batch: 40, Epoch: 2, Loss: 1.50
Batch: 50, Epoch: 2, Loss: 1.53
Epoch [2/10], Avg Loss: 1.4751
*** Validation Accuracy: 42.61% ***
Batch: 10, Epoch: 3, Loss: 1.55
Batch: 20, Epoch: 3, Loss: 1.22
Batch: 30, Epoch: 3, Loss: 1.36
Batch: 40, Epoch: 3, Loss: 1.43
Batch: 50, Epoch: 3, Loss: 1.94
Epoch [3/10], Avg Loss: 1.3769
*** Validation Accuracy: 45.57% ***
Batch: 10, Epoch: 4, Loss: 0.96
Batch: 20, Epoch: 4, Loss: 1.39
Batch: 30, Epoch: 4, Loss: 1.22
Batch: 40, Epoch: 4, Loss: 1.16
Batch: 50, Epoch: 4, Loss: 1.22
Epoch [4/10], Avg Loss: 1.2856
*** Validation Accuracy: 49.22% ***
Batch: 10, Epoch: 5, Loss: 1.12
Batch: 20, Epoch: 5, Loss: 1.41
Batch: 30, Epoch: 5, Loss: 

## Model 3: Transfer Learning with EfficientNet


In [20]:
model = models.efficientnet_b0(weights='DEFAULT')
model.classifier[1].in_features

1280

In [21]:
class CarClassifierEfficientNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = models.efficientnet_b0(weights='DEFAULT')
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        in_features = self.model.classifier[1].in_features
        
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, num_classes)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x 

In [22]:
model = CarClassifierEfficientNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
train_model(model, criterion, optimizer, epochs=10)
#all_labels, all_predictions = train_model(model, criterion, optimizer, epochs=10)

Batch: 10, Epoch: 1, Loss: 1.56
Batch: 20, Epoch: 1, Loss: 1.60
Batch: 30, Epoch: 1, Loss: 1.35
Batch: 40, Epoch: 1, Loss: 1.36
Batch: 50, Epoch: 1, Loss: 1.33
Epoch [1/10], Avg Loss: 1.4807
*** Validation Accuracy: 54.43% ***
Batch: 10, Epoch: 2, Loss: 1.28
Batch: 20, Epoch: 2, Loss: 1.19
Batch: 30, Epoch: 2, Loss: 1.15
Batch: 40, Epoch: 2, Loss: 1.06
Batch: 50, Epoch: 2, Loss: 1.00
Epoch [2/10], Avg Loss: 1.1281
*** Validation Accuracy: 57.57% ***
Batch: 10, Epoch: 3, Loss: 1.15
Batch: 20, Epoch: 3, Loss: 1.00
Batch: 30, Epoch: 3, Loss: 1.17
Batch: 40, Epoch: 3, Loss: 0.71
Batch: 50, Epoch: 3, Loss: 0.97
Epoch [3/10], Avg Loss: 0.9970
*** Validation Accuracy: 60.35% ***
Batch: 10, Epoch: 4, Loss: 0.92
Batch: 20, Epoch: 4, Loss: 1.03
Batch: 30, Epoch: 4, Loss: 0.97
Batch: 40, Epoch: 4, Loss: 0.73
Batch: 50, Epoch: 4, Loss: 0.88
Epoch [4/10], Avg Loss: 0.9515
*** Validation Accuracy: 61.74% ***
Batch: 10, Epoch: 5, Loss: 0.78
Batch: 20, Epoch: 5, Loss: 0.86
Batch: 30, Epoch: 5, Loss: 0

([np.int64(1),
  np.int64(2),
  np.int64(2),
  np.int64(4),
  np.int64(2),
  np.int64(2),
  np.int64(1),
  np.int64(2),
  np.int64(0),
  np.int64(3),
  np.int64(4),
  np.int64(1),
  np.int64(3),
  np.int64(2),
  np.int64(1),
  np.int64(1),
  np.int64(5),
  np.int64(0),
  np.int64(3),
  np.int64(4),
  np.int64(3),
  np.int64(0),
  np.int64(2),
  np.int64(5),
  np.int64(1),
  np.int64(0),
  np.int64(1),
  np.int64(4),
  np.int64(0),
  np.int64(0),
  np.int64(2),
  np.int64(0),
  np.int64(1),
  np.int64(4),
  np.int64(3),
  np.int64(0),
  np.int64(2),
  np.int64(5),
  np.int64(2),
  np.int64(1),
  np.int64(0),
  np.int64(3),
  np.int64(0),
  np.int64(2),
  np.int64(4),
  np.int64(2),
  np.int64(5),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(0),
  np.int64(3),
  np.int64(0),
  np.int64(2),
  np.int64(2),
  np.int64(2),
  np.int64(4),
  np.int64(2),
  np.int64(0),
  np.int64(2),
  np.int64(1),
  np.int64(4),
  np.int64(1),
  np.int64(0),
  np.int64(3),
  np.int64(1),
  np.int64

## Model 4: Transfer Learning with ResNet

In [23]:
# Load the pre-trained ResNet model
class CarClassifierResNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.model = models.resnet50(weights='DEFAULT')
        # Freeze all layers except the final fully connected layer
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze layer4 and fc layers
        for param in self.model.layer4.parameters():
            param.requires_grad = True            
            
        # Replace the final fully connected layer
        self.model.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(self.model.fc.in_features, num_classes)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [24]:
model = CarClassifierResNet(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

labels, predictions = train_model(model, criterion, optimizer, epochs=10)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\prati/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:18<00:00, 5.44MB/s]


Batch: 10, Epoch: 1, Loss: 0.92
Batch: 20, Epoch: 1, Loss: 0.66
Batch: 30, Epoch: 1, Loss: 0.70
Batch: 40, Epoch: 1, Loss: 0.51
Batch: 50, Epoch: 1, Loss: 0.66
Epoch [1/10], Avg Loss: 0.8768
*** Validation Accuracy: 69.22% ***
Batch: 10, Epoch: 2, Loss: 0.57
Batch: 20, Epoch: 2, Loss: 0.70
Batch: 30, Epoch: 2, Loss: 0.46
Batch: 40, Epoch: 2, Loss: 0.27
Batch: 50, Epoch: 2, Loss: 0.36
Epoch [2/10], Avg Loss: 0.4715
*** Validation Accuracy: 76.17% ***
Batch: 10, Epoch: 3, Loss: 0.54
Batch: 20, Epoch: 3, Loss: 0.40
Batch: 30, Epoch: 3, Loss: 0.25
Batch: 40, Epoch: 3, Loss: 0.39
Batch: 50, Epoch: 3, Loss: 0.19
Epoch [3/10], Avg Loss: 0.3424
*** Validation Accuracy: 78.43% ***
Batch: 10, Epoch: 4, Loss: 0.14
Batch: 20, Epoch: 4, Loss: 0.19
Batch: 30, Epoch: 4, Loss: 0.26
Batch: 40, Epoch: 4, Loss: 0.31
Batch: 50, Epoch: 4, Loss: 0.17
Epoch [4/10], Avg Loss: 0.2549
*** Validation Accuracy: 76.17% ***
Batch: 10, Epoch: 5, Loss: 0.32
Batch: 20, Epoch: 5, Loss: 0.15
Batch: 30, Epoch: 5, Loss: 0

#### I ran hyperparameter tunning in another notebook and figured that the best parameters for resnet models are (1) Dropout rate = 0.2 (2) Learning Rate = 0.005

So now let's train the model once again with these best parameters

In [26]:
model = CarClassifierResNet(num_classes=num_classes, dropout_rate=0.2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.005)

labels, predictions = train_model(model, criterion, optimizer, epochs=10)

Batch: 10, Epoch: 1, Loss: 1.04
Batch: 20, Epoch: 1, Loss: 0.99
Batch: 30, Epoch: 1, Loss: 0.69
Batch: 40, Epoch: 1, Loss: 0.64
Batch: 50, Epoch: 1, Loss: 0.70
Epoch [1/10], Avg Loss: 0.8671
*** Validation Accuracy: 72.17% ***
Batch: 10, Epoch: 2, Loss: 0.38
Batch: 20, Epoch: 2, Loss: 0.43
Batch: 30, Epoch: 2, Loss: 0.50
Batch: 40, Epoch: 2, Loss: 0.58
Batch: 50, Epoch: 2, Loss: 0.63
Epoch [2/10], Avg Loss: 0.4886
*** Validation Accuracy: 71.30% ***
Batch: 10, Epoch: 3, Loss: 0.26
Batch: 20, Epoch: 3, Loss: 0.23
Batch: 30, Epoch: 3, Loss: 0.17
Batch: 40, Epoch: 3, Loss: 1.00
Batch: 50, Epoch: 3, Loss: 0.50
Epoch [3/10], Avg Loss: 0.3883
*** Validation Accuracy: 78.61% ***
Batch: 10, Epoch: 4, Loss: 0.30
Batch: 20, Epoch: 4, Loss: 0.22
Batch: 30, Epoch: 4, Loss: 0.31
Batch: 40, Epoch: 4, Loss: 0.12
Batch: 50, Epoch: 4, Loss: 0.22
Epoch [4/10], Avg Loss: 0.2710
*** Validation Accuracy: 80.00% ***
Batch: 10, Epoch: 5, Loss: 0.24
Batch: 20, Epoch: 5, Loss: 0.31
Batch: 30, Epoch: 5, Loss: 0

# 5. Model Evaluation using Confusion Matrix and Classification Report

In [27]:
from sklearn.metrics import classification_report

report = classification_report(labels, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.91      0.82       120
           1       0.75      0.68      0.71       103
           2       0.91      0.79      0.85       117
           3       0.74      0.84      0.78        74
           4       0.63      0.69      0.66        77
           5       0.86      0.68      0.76        84

    accuracy                           0.77       575
   macro avg       0.77      0.76      0.76       575
weighted avg       0.78      0.77      0.77       575



In [28]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib import pyplot as plt

conf_matrix = confusion_matrix(labels, predictions, labels=np.arange(num_classes))
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title("Confusion Matrix for Vehicle Damage Classification")
plt.show()

NameError: name 'class_names' is not defined

# 6. Save the Model

In [ ]:
torch.save(model.state_dict(), 'saved_model.pth')